# BPL_CHO_Fedbatch - demo

Author: Jan Peter Axelsson

This notebook deals with CHO fedbatch cultivation and recombinant protein production is included. First we make a check of the model by comparing a simulation result with corresponding published diagram. Then we take a closer look at the start-up strategy to keep the by-product formation low. After that we investigate at a whole cultivation and see the impact of feeding strategy on both cell growth and protein production where a trade-off is needed in this case. 

The model used takes its inspiration from the microbial bottleneck models as described in the original papers [1] and [2] and reformulated and studied in [3]. The laboratory cultures used for model validation in [1] did produce MAb (against part of IgG) but no MAb-data was presented. The paper focus on viable and non-viable cell concentrations only. The original model is in section 5 expanded with the classical empirical Luedeking-Piret model recombinant protein production, see chapter 5 in [4]. In this way can get more insight into choice of feeding profile. 

Interaction with the compiled model as FMU is mainly through the simplified commands: par(), init(), newplot(), simu() etc. The last simulation is always available in the workspace and called 'sim_res'. The command describe() brings mainly up description infomration from the actual Modelica code from the FMU but is complemented with information given in the dedicated Python setup-file.

The idea here is to demonstrate how simulations and varyiing conditions can provide some process insight that can support the experimental work. I hope that at the end of this session you are ready to formulate your own questions you want to address with simulations - and you can just go on in this notebook! Just press the field "+Code" in the upper left part of notebook interface and you get a new "cell" where you write your own code. You can copy and paste from cells above using ctrl-c and ctrl-v as usual and edit the cell. When your are ready to execute the cell just press the "play button" to the left in the cell or press shift-enter as in "ordinary" Jupyter notebooks.

After a session you may want to save your own notebook. That you can do on your Google Drive account and I refer to Colab instructions for how to do this. It is easy.

Good luck!

In [1]:
run -i BPL_CHO_fedbatch_explore.py

ModuleNotFoundError: No module named 'pyfmi'

In [2]:
plt.rcParams['figure.figsize'] = [30/2.54, 24/2.54]

## 1 About the process model

We can get information about the process and liquid phase by the command describe(). Here is no gas-phase included. This command can also be used to bring up information about a specific variable or parameter. However, you should use describe() after a simulation to get the valued used during the simulation.

In [3]:
process_diagram()

NameError: name 'process_diagram' is not defined

In [ ]:
describe('culture'); print(); #describe('liquidphase')

The molecular weight of the recombinant protein (MAb) is somewhat arbitrarly chosen and the value not used in the simulations.

## 2 Simulation reproducing the original paper

The simulation below reproduce diagrams in Figure 5 in the original paper. There are several simulation in the paper showing how well the model describe different experiments and here I just choose one of them.



In [ ]:
# Data from Table 1 and 2 for experiment 4 shown in Figure 5 in paper [1]
# -culture parameters taken from Table 5 identified parameters for cultures 1,2,and 3

# Initial process conditions
V_start=0.30
init(V_start=V_start, VXv_start=V_start*0.172, VXd_start=V_start*0.020) 
init(VG_start=V_start*17.83, VGn_start=V_start*3.74, VL_start=V_start*0.12, VN_start=V_start*0.24)

# Feeding
Feed=0.1/24
par(G_in=15, Gn_in=9.3)
par(t0=0, F0=0, t1=63.5, F1=Feed, t2=300, F2=Feed)

# Simulation
newplot(title='Figure 5 - Fedbatch cultivation') 
simu(200)

**Comment:** The simulation results looks very similar to the published diagram Figure 5 in [1]. The model pass this quality check.

## 3 Simulation of different start-up feeding strategies

In [ ]:
DeltaVG = 15*0.1*0.4
print(DeltaVG)

In [ ]:
DeltaVG_0=(1-0.4)*0.30*17.8
print(DeltaVG_0)

In [ ]:
describe('bioreactor.inlet[1].F')

In [ ]:
# Figur 5
V_start=0.30
init(V_start=V_start, VXv_start=V_start*0.172, VXd_start=V_start*0.020) 
init(VG_start=V_start*17.83, VGn_start=V_start*3.74, VL_start=V_start*0.12, VN_start=V_start*0.24)

# Feeding
Feed=0.1/24
par(G_in=15, Gn_in=9.3)
par(t0=0, F0=0, t1=63.5, F1=Feed, t2=300, F2=Feed)

newplot(title='Figure 5 - Fedbatch cultivation') 
simu(120)

init(VG_start=0.35*V_start*17.83, VGn_start=0.35*V_start*3.74)
par(t0=0, F0=0, t1=40.0, F1=0.5*Feed, t2=63.5, F2=Feed,t3=300, F3=Feed)
simu(120)

init(VG_start=0.35*V_start*17.83, VGn_start=0.35*V_start*3.74)
par(t0=0, F0=0, t1=40.0, F1=0.5*Feed, t2=63.5, F2=0.7*Feed, t3=88.0, F3=Feed, t4=300, F4=Feed)
simu(120)

# Reset time table to avoid problems below
par(t1=1001, t2=1002, t3=1003, t4=1004, t5=1005, t6=1006)

**Comment:** We see that starting the feed a day erlier at lower rate and then increase decreaes lactate formation to half, while the cell conentation is just slightlly lower. With a more careful design of the feedprofile the ammonia formation can be decreased more than shown here.

## 4 Simulation of optimal feed profile for cell growth

At the end of the original paper section 5 in [1], the derived model is used to find an optimal feeding profile for high final cell concentration. It is stated that protein productivity is assumed to be mainly positively growth associated and therefore optimization of cell concentration is very similar to optimization of protein product. The optimization of feed profile is done with differnt structures of the feed profile. All of them have a start-time and all of them has a fixed amount of substrate and concentrations in the media are also the same. 

* The first optimzation is for a feed profile similar to the experimental, i.e. after start the feed rate remains constant throughout the cultivation. Thus the start time and the actual feed rate are optimized. The result was that the start time was about the same as expermentally but the feed rate was 50% higher, see Figure 7 and Figure 10 in [1]. 

* The second optimization is for a feed profile with not just one increase but three steps of increase of feed rate. The results is a somwehat higher final cell concentration, see Figure 11.

* The third optimization is for a feed profile with five steps of increase of feed rate. The results is a slightly higher final cell concentration than for three steps, see Figure 12.

* The fourth optimization is for a feed profie with continuous exponential increase of the reed rate. The result is a bit higher final concentration than the previous with five steps, see Figure 13 but not shown in the figure below.

Below we just show the results of the original experimetnal cultivation, compared with results from three and five steps.
It is possible to do the optimization in Python with the FMU, but we save that for a future notebook.

In [ ]:
# Culture parameters taken from Table 5 identified parameters for cultures 1,2,and 3

# Data chosen
V_start=0.35
init(V_start=V_start, VXv_start=V_start*0.20, VXd_start=V_start*0.0)
init(VG_start=V_start*18.0, VGn_start=V_start*2.4, VL_start=V_start*0, VN_start=V_start*0)

# Feeding n=1 - experimental and lower feed rate
par(G_in=15, Gn_in=4.0)
par(t0=0, F0=0, t1=49, F1=0.00417)
par(t2=1002, t3=1003, t4=1004, t5=1005)

# Simulation
newplot(title='Figure 12 - Fedbatch with optimal step-wise feed') 
simu(125)

# Feeding n=1
par(G_in=15, Gn_in=4.0)
par(t0=0, F0=0, t1=52, F1=0.00625)
par(t2=1002, t3=1003, t4=1004, t5=1005, t6=1006)

# Simulation
simu(125)

# Feeding n=3
par(G_in=15, Gn_in=4.0)
par(t0=0, F0=0, t1=52, F1=0.002, t2=74, F2=0.0045, t3=98.0, F3=0.010) 
par(t4=99.0, F4=0.010, t5=106, F5=0.010, t6=150, F6=0.010)

# Simulation
simu(125)

# Feeding n=5
par(G_in=15, Gn_in=4.0)
par(t0=0, F0=0, t1=52, F1=0.0012, t2=64, F2=0.0020, t3=78.0, F3=0.0040) 
par(t4=92.0, F4=0.0080, t5=106, F5=0.012, t6=150, F6=0.012)

# Simulation
simu(125)

# Reset feeding parameters since the table need time in strict increasing value
par(t3=1004, t4=1005, t5=1006, t6=1007)

**Comment:** We see that that already the better tuned constant feed rate (dahsed) compared to the experimental (solid) gives higher final cell concnetration. 

Breaking up the constant feed rate in three (dotted) and five (dash-dotted) steps with a more gradual increase of the feed rate gives even higher final cell concentration. The difference between n=3 and n=5 is small. The change to continuous exponential feed is even smaller and not shown here.

The results shown here are similar to what is presented in Table 7 in [1] but our simulation are slightly longer and here are small differences in the final cel concentration too. The qualitative result is the same though. The difference we see to the result in the original paper is most likely due to the fact that we here use the full model with 17 parameters while in the paper they have reduced the model to 15 parameters for the optimization work. 

## 5 Simulation of different feed profiles to increase recombinant protein production

In this section we take a closer look at recombinant protein production. The original model is extended with the empirical model for specific protein production, see chapter 5 in [4]

$$
q_P = \alpha \cdot \mu + \beta
$$

Here we choose a negative value of growth-associated protein production production $\alpha$ while keeping the non-growth associated $\beta$ positive. The culture produced recombinant protein in the form of monoclonal antibodies for a specific IgG1 molecule,see section 2 in [1]. However, no experimental results were given. The only information we have is that feed rate was kept constant at a low level during fedbatch production and this choice indicates that the the growth-associated protein production is negative. The consequence of this observation for the feed profile we take a look at there by simulation.

In [ ]:
# Slide 3
newplot('CHO fedbatch cultivation - protein expression', plotType='Textbook_3')

# Data from Table 1 and 2 for experiment 3
V_start=0.35
init(V_start=V_start, VXv_start=V_start*0.29, VXd_start=V_start*0.010)
init(VG_start=V_start*17.17, VGn_start=V_start*3.02, VL_start=V_start*1.12, VN_start=V_start*0.29)

# Feeding
Feed=0.1/24
par(G_in=15, Gn_in=9.3)
par(t0=0, F0=0, t1=35, F1=Feed, t2=100, F2=Feed, t3=300, F3=Feed)

# Culture parameters
par(alpha=-1.0, beta=0.01)

# Simulation
simu(200)
par(t2=100, F2=0.7*Feed, t3=300, F3=0.7*Feed); simu(200)
par(F2=Feed, F3=Feed)

**Comment:** The simulation results show that actually a decease in the feed rate can lead to an increase in recombinant protein produced, although the cell concentration is a bit lower. This is a result due to the fact that growth-associated protein production here is set to a negative value. The main point is that the model can actually capture this phenomena. 

## 6 Summary

In short we have done the following:

* The model was checked by comparing the simulation results with one of the published diagrams [1].

* The common startup-procedure with 3 days batch cultivation can be questioned. We found that by shorten it to 2 days, and giving smaller feed rate day 3, byproduct formation can be kept lower at the prize of just a bit lower cell concentration. Similar idea was shown in section 2.1 in [3].

* In the original paper the experimental feeding strategy was to keep the substrate feed at a constant lower level. The authors made a point of that the optimal feeding strategy should be exponential for maximal cell production. This is an insight derived from the bottle-neck model and they showed that through simulation optimization [1]. However, there was no experimental support to confirm the results. The optimal cell growth feedprofile simulation was just reproduced here.

* To optimize recombinant protein production we must include production in the model. Here we do that with the empirical model that distinguish between growth-associated and non-growth-associated protein production, see chapter 5 in [4]. For a class of CHO-processes the recombinant protein productivity is acutally negatively affected by cell growth. Simulation of the original model extended with such a protein production model shows that keeping the substrate feed rate constant as the cell culture grows, giving less and less feed per cell, actually can give higher protein production than an increaeing feed rate. Simulation confirms this idea. The results gives some possible background to why the constant feed rate was used experimetnally in the original paper [1].


## 7 References

[1] Amribt, Z., Niu, H. and Bogaerts P.: "Macroscopic modelling of overflow metabolism and model based optimization of hybridoma cell fed-batch cultures.", Biochem. Eng. Journal, 2013.

[2] Niu,H., Amribt, Z., Fickers, P., Tan, W. and Bogaerts P.: "Metabolic pathway analysis and reduction for mammalian cell cultures - towards macroscopic modelling", Chem. Eng. Science, 2013.

[3] Axelsson, J. P.: "Simplified model of CHO-cultivation in Bioproces Library for Modelica - some experience", conference paper 22nd NPCW Lyngby, Denmark, August 22-23, 2019.

[4] Hu, W-S: "Cell culture bioprocess engineering", 2nd edition, CRC Press, 2020.

## Appendix

In [ ]:
disp('_start', mode='long')

In [ ]:
disp('culture')

In [ ]:
describe('mu')

In [ ]:
# List of components in the process setup and also a couple of other things like liquidphase and gasphase etc
describe('parts')

In [ ]:
describe('MSL')

In [ ]:
system_info()